In [ ]:
import pandas as pd
import snowflake.connector
from datetime import datetime, timedelta

# Connect using saved profile
conn = snowflake.connector.connect(
    connection_name="DELIVEROO",
    database="production",  # ← your database
    schema="denormalised"   # ← your schema
)

# Define your Python variables
start_date = '2025-04-11'
comms_date = '2025-04-11'
end_date = (datetime.strptime(start_date, '%Y-%m-%d') + timedelta(days=14)).strftime('%Y-%m-%d')
country_list = "('UK', 'Ireland')"

# Inject into SQL string (use f-string carefully to avoid SQL injection)
query = f"""
SELECT 
  A.DRIVER_ID,
  CLUSTER_NAME AS Cluster,
  CITY_NAME AS City,
  COUNT(OA_STATUS) ORDER_COUNT,
  SUM(CASE WHEN OA_STATUS = 'DELIVERED' AND TO_DATE(LOCAL_TIME_OA_CREATED_AT) BETWEEN TO_DATE('{start_date}') AND DATEADD(DAY, (7 - DAYOFWEEK(TO_DATE('{start_date}'))), TO_DATE('{start_date}')) THEN 1 ELSE 0 END) AS Orders_Week_1,
  SUM(CASE WHEN OA_STATUS = 'DELIVERED' AND TO_DATE(LOCAL_TIME_OA_CREATED_AT) BETWEEN DATEADD(DAY, (8 - DAYOFWEEK(TO_DATE('{start_date}'))), TO_DATE('{start_date}')) AND DATEADD(DAY, (14 - DAYOFWEEK(TO_DATE('{start_date}'))), TO_DATE('{start_date}')) THEN 1 ELSE 0 END) AS Orders_Week_2,
  SUM(CASE WHEN OA_STATUS = 'DELIVERED' AND TO_DATE(LOCAL_TIME_OA_CREATED_AT) BETWEEN DATEADD(DAY, (15 - DAYOFWEEK(TO_DATE('{start_date}'))), TO_DATE('{start_date}')) AND TO_DATE('{end_date}') THEN 1 ELSE 0 END) AS Orders_Week_3,
  CASE WHEN A.DRIVER_ID IN (
        SELECT driver_id
        FROM braze.rider_engagement
        WHERE 
          sent_at >= CONVERT_TIMEZONE('Europe/London', 'UTC', TO_TIMESTAMP('{comms_date}')) AND
          sent_at < CONVERT_TIMEZONE('Europe/London', 'UTC', DATEADD(day, 1, TO_TIMESTAMP('{comms_date}')))
          AND campaign_canvas_name = 'rid-multi-reac-rec-gen-global-all-all-week45-20241106-reactivationautomation' 
          AND canvas_step_name = 'Email_1_Awareness'
      ) THEN 1 ELSE 0 END AS In_Comms
FROM production.denormalised.denormalised_assignment A
WHERE 
  COUNTRY_NAME IN {country_list}
  AND OA_STATUS = 'DELIVERED'  
  AND TO_DATE(LOCAL_TIME_OA_CREATED_AT) BETWEEN TO_DATE('{start_date}') AND TO_DATE('{end_date}')
  AND A.DRIVER_ID IS NOT NULL
  AND A.DRIVER_ID IN (
    '106390','505713','849881','934540','929023','940260','922071','758632','934401','795804','934617','843983','695424','944246','934726','505525','417317','900354','944148','886855','697658','879482','175548','926218','979392','616815','944845','829383','404096','471105','523572','200440','675381','850164','939888','811340','959681','617326','931442','940141','688207','836929','694661','814729','932653','939889','497003','782239','863716','917121','783400','623310','864960','837777','878699','745115','949024','455809','873527','927181','225373','948455','558869','546019','927935','938134','934145','462115','908205','940391','849786','450554','586436','935359','430713','964470','965251','854144','932956','781277','878550','381418','969328','773001','936429','976208','748880','394458','929840','938694','600394','182757','969060','844523','976210','976774','979925','144989','775592','117296','956776','355307','938565','848518','571979','178883','925982','932795','343000','770604','929999','646336','950700','346267','544298','829117','340871','931331','422098','938401','906414','879322','571681','946153','821606','828535','846721','334278','397703','357645','963288','481727','942341','425864','749512','817942','954222','340894','935173','822307','845380','749505','168384','878843','733547','828501','846472','929981','963742','433264','532515','827658','923870','931624','562468','672212','863724','342827','698832','491446','528197','936858','669548','410379','408141','811201','944439','934270','939135','546690','873845','406611','938513','508649','946950','947626','807924','818962','869476','935697','677054','977889','696818','793726'
  )
GROUP BY 
  A.DRIVER_ID, CITY_NAME, CLUSTER_NAME, In_Comms
ORDER BY 
  ORDER_COUNT DESC, A.DRIVER_ID, CITY_NAME, CLUSTER_NAME
"""

# Run the query
df = pd.read_sql(query, conn)

# Preview
df.head()


Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://accounts.google.com/o/saml2/idp?idpid=C01jnk96c&SAMLRequest=nZLBbuIwEIZfJfKeEzspsMUioABliUQXBAG1vRnHUBfHztoOad9%2BHQJS99Ae9mDJsv%2BZb2b%2BGYzeC%2BGdmTZcyRiEAQIek1TlXB5jsM1m%2Fj3wjCUyJ0JJFoMPZsBoODCkECVOKvsq1%2BxPxYz1XCJpcPMRg0pLrIjhBktSMIMtxZvkcYGjAGFiDNPW4cA1JDfcsV6tLTGEdV0H9V2g9BFGCCGI%2BtCpGskP8AlRfs8otbKKKnELeXc9fYEIIeo0CKdwhNU1cMxlO4LvKPtWZPA8y1b%2BarnJgJfcupsoaaqC6Q3TZ07Zdr1oCzCugunDIt09rJfLgFU%2BZdJqIvwwMFLVB0FOjKqirKxLHbgbPLAcCnXkbmDpNAbliecRGSf7Lkr246hXLHq%2FluREnjO75jSd7efzp%2B55oZ92b4%2FZYUuBt7vZGzX2psZULJWNqdY9oajro44f9jLUx50I30XBz979C%2FCmzlQuib1E3ionlKpKWhMclToKdqlPwcaRCPK8HLnD83iCwjd56vcoaDcFX5h6%2BF%2F9D%2BDnFNfN%2B%2B3MSKcrJTj98GZKF8R%2B7VUYhJcXnvuHixSzgnCR5LlmxjjPhFD1RDNi3YJbXTEAhy313xUf%2FgU%3D&RelayState=ver%3A1-h

/var/folders/5t/7fmzl0lx2ds7q_hszd4f9zt00000gq/T/ipykernel_69768/465423203.py:53: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,DRIVER_ID,CLUSTER,CITY,ORDER_COUNT,ORDERS_WEEK_1,ORDERS_WEEK_2,ORDERS_WEEK_3,IN_COMMS
0,965251,GOD-GUI-WOK,London,44,22,22,0,1
1,523572,CIR,Cheltenham,33,23,10,0,1
2,600394,GOD-GUI-WOK,London,31,16,15,0,1
3,869476,WTH,Chelmsford,23,23,0,0,1
4,922071,BAS,Reading,18,10,8,0,1


In [4]:
df


,DRIVER_ID,CLUSTER,CITY,ORDER_COUNT,ORDERS_WEEK_1,ORDERS_WEEK_2,ORDERS_WEEK_3,IN_COMMS
0,965251,GOD-GUI-WOK,London,44,22,22,0,1
1,523572,CIR,Cheltenham,33,23,10,0,1
2,600394,GOD-GUI-WOK,London,31,16,15,0,1
3,869476,WTH,Chelmsford,23,23,0,0,1
4,922071,BAS,Reading,18,10,8,0,1
5,929023,BAS,Reading,15,10,5,0,1
6,934145,GOD-GUI-WOK,London,15,12,3,0,1
7,821606,NWB,Reading,10,10,0,0,1
8,869476,CLC,Colchester,9,7,2,0,1
9,617326,CIR,Cheltenham,7,7,0,0,0
